In [ ]:
import pandas as pd

df1 = pd.read_csv("variables/cloudcover.csv")
df2 = pd.read_csv("variables/humidity.csv")
df3 = pd.read_csv("variables/windspeed.csv")
df4 = pd.read_csv("variables/cloudcover.csv")
df5 = pd.read_csv("variables/ndvi.csv")
df6 = pd.read_csv("variables/forestcover.csv")
df7 = pd.read_csv("variables/tmax.csv")
df8 = pd.read_csv("variables/tmin.csv")
df9 = pd.read_csv("variables/elevation.csv")
df10 = pd.read_csv("variables/rain.csv")
df11 = pd.read_csv("variables/population.csv")
df12 = pd.read_csv("variables/firecount.csv")

df1=df1.drop(0)
df2=df2.drop(0)
df3=df3.drop(0)
df4=df4.drop(0)
df5=df5.drop(0)
df6=df6.drop(0)
df7=df7.drop(0)
df8=df8.drop(0)
df9=df9.drop(0)
df10=df10.drop(0)
df11=df11.drop(0)
df12=df12.drop(0)

import pandas as pd

# Initialize an empty dataframe to hold the combined data
combined_data = pd.DataFrame()

# Iterate through the dataframes and preprocess them before merging
for i in range(1, 13):  # Assuming you have 8 dataframes named df1, df2, ..., df8
    df_name = 'df' + str(i)
    if df_name in globals():  # Check if the dataframe exists
        df = globals()[df_name]
        print(f"Processing {df_name}")

        # Convert 'Lat' and 'Lon' columns to numeric values
        df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
        df['Lon'] = pd.to_numeric(df['Lon'], errors='coerce')

        # Remove rows with non-numeric values in 'Lat' and 'Lon' columns
        df = df.dropna(subset=['Lat', 'Lon'])

        if not combined_data.empty:
            combined_data = pd.merge(combined_data, df, on=['Lat', 'Lon', 'Date'], how='outer', sort=True)
        else:
            combined_data = df.copy()


combined_data = combined_data.dropna()

# Reset index
combined_data.reset_index(drop=True, inplace=True)

# Save the combined dataframe to a CSV file
combined_data.to_csv("combined_file_all_grids.csv", index=False)


# Determining top 30 griids

In [ ]:
import pandas as pd

# 1. Read the fire data
fire_data = pd.read_csv('variables/firecount.csv')  # Replace 'fire_data.csv' with the path to your file

# Group the data by grid cell (latitude and longitude) and sum the fire counts for each grid cell across all days
grouped_data = fire_data.groupby(['Lat', 'Lon'])['fire_count'].sum()

# Calculate the total number of fire counts in the dataset
total_counts = grouped_data.sum()

# Sort the grid cells based on the summed fire counts
sorted_cells = grouped_data.sort_values(ascending=False)

# Calculate the threshold for the top 10% of grid cells
top_30_percent_threshold = sorted_cells.quantile(0.70)

# Filter the grid cells to retain only those that exceed the threshold
top_30_percent_cells = sorted_cells[sorted_cells >= top_30_percent_threshold]


# Save the top 10% grid cells to a CSV file
top_30_percent_cells.to_csv('final_top_30_grids.csv', header=True)

# Final Combined data of top 30 grids

In [ ]:
import pandas as pd

df1 = pd.read_csv('combined_file_all_grids.csv')
df2 = pd.read_csv('final_top_30_grids.csv')

# Merge the first dataframe with the unique Lat and Lon values from the second dataframe
merged_df = pd.merge(combined_data, df2, on=['Lat', 'Lon'], how='inner')

# Display the resulting dataframe with matching Lat and Lon values
sorted_df = merged_df.sort_values(by=['Lat', 'Lon'])

# Save the sorted dataframe to a CSV file
sorted_df.to_csv('final_top_30_grids_combined.csv', index=False)